In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emg-test-1/emg_test/down_test/5_18_08_09.csv
/kaggle/input/emg-test-1/emg_test/down_test/4_18_08_04.csv
/kaggle/input/emg-test-1/emg_test/down_test/6_18_08_16.csv
/kaggle/input/emg-test-1/emg_test/right_test/12_18_09_07.csv
/kaggle/input/emg-test-1/emg_test/right_test/11_18_09_00.csv
/kaggle/input/emg-test-1/emg_test/right_test/10_18_08_52.csv
/kaggle/input/emg-test-1/emg_test/left_test/9_18_08_40.csv
/kaggle/input/emg-test-1/emg_test/left_test/8_18_08_33.csv
/kaggle/input/emg-test-1/emg_test/left_test/7_18_08_26.csv
/kaggle/input/emg-test-1/emg_test/up_test/1_18_07_38.csv
/kaggle/input/emg-test-1/emg_test/up_test/3_18_07_52.csv
/kaggle/input/emg-test-1/emg_test/up_test/2_18_07_44.csv
/kaggle/input/emg-fin-csv/emg_dataset_final/right/right46.csv
/kaggle/input/emg-fin-csv/emg_dataset_final/right/right3.csv
/kaggle/input/emg-fin-csv/emg_dataset_final/right/right25.csv
/kaggle/input/emg-fin-csv/emg_dataset_final/right/right76.csv
/kaggle/input/emg-fin-csv/emg_dataset_final/r

In [2]:
import h5py

# Define the paths to your HDF5 files
file_paths = [
    "/kaggle/input/emg-fin/emg_signals_up.h5",
   
]

# Function to get dimensions of HDF5 dataset
def get_dataset_dimensions(file_path):
    with h5py.File(file_path, 'r') as file:
        dataset = file['emg_signal_up_1']  # Replace 'dataset_name' with the actual dataset name in your HDF5 file
        return dataset.shape

# Iterate over each file and print its dimensions
for path in file_paths:
    dimensions = get_dataset_dimensions(path)
    print(f"Dimensions of {path}: {dimensions}")


Dimensions of /kaggle/input/emg-fin/emg_signals_up.h5: (8, 1000, 7)


In [4]:
import h5py
import torch
from torch.utils.data import Dataset, DataLoader

class EMGDataset(Dataset):
    def __init__(self, file_paths):
        self.all_features = []
        self.all_labels = []

        for file_path in file_paths:
            with h5py.File(file_path, 'r') as file:
                for key in file.keys():
                    data = torch.tensor(file[key][:] ,  dtype=torch.float32)  # Load data into PyTorch tensor
                    features = data[:, :, :6]  # Extract features (first 6 columns)
                    labels = data[:, :, 6]     # Extract labels (7th column)

                    self.all_features.append(features)
                    self.all_labels.append(labels)

        # Concatenate all features and labels
        self.all_features = torch.cat(self.all_features, dim=0)
        self.all_labels = torch.cat(self.all_labels, dim=0)

    def __len__(self):
        return len(self.all_features)

    def __getitem__(self, idx):
        return self.all_features[idx], self.all_labels[idx] # Convert labels to long (int) type


# Define the paths to your HDF5 files
file_paths = [
    "/kaggle/input/emg-fin/emg_signals_up.h5",
    "/kaggle/input/emg-fin/emg_signals_left.h5",
    "/kaggle/input/emg-fin/emg_signals_right.h5",
    "/kaggle/input/emg-fin/emg_signals_down.h5"
]

# Create dataset
dataset = EMGDataset(file_paths)

# Split the dataset into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

# Create DataLoader for train, validation, and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example: Iterating over the train loader
for features, labels in train_loader:
    print("Train Data Shapes:", features.shape, labels.shape)
    break  # Breaking after one iteration for demonstration

# Example: Iterating over the validation loader
for features, labels in val_loader:
    print("Validation Data Shapes:", features.shape, labels.shape)
    break  # Breaking after one iteration for demonstration

# Example: Iterating over the test loader
for features, labels in test_loader:
    print("Test Data Shapes:", features.shape, labels.shape)
    break  # Breaking after one iteration for demonstration


Train Data Shapes: torch.Size([32, 1000, 6]) torch.Size([32, 1000])
Validation Data Shapes: torch.Size([32, 1000, 6]) torch.Size([32, 1000])
Test Data Shapes: torch.Size([32, 1000, 6]) torch.Size([32, 1000])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.input_dim = input_dim
        self.W = nn.Linear(input_dim, input_dim)
        self.v = nn.Linear(input_dim, 1, bias=False)

    def forward(self, encoder_outputs):
        energy = torch.tanh(self.W(encoder_outputs))
        attention_scores = self.v(energy).squeeze(dim=-1)
        attention_weights = F.softmax(attention_scores, dim=-1)
        context_vector = torch.sum(encoder_outputs * attention_weights.unsqueeze(-1), dim=1)
        return context_vector, attention_weights


class EMGAttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bidirectional, num_classes):
        super(EMGAttentionModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_classes = num_classes

        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2, bidirectional=bidirectional)
        self.attention = Attention(hidden_size * 2 if bidirectional else hidden_size)
        self.fc1 = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, 256)  # Increase output size
        self.fc2 = nn.Linear(256, 128)  # New fully connected layer
        self.fc3 = nn.Linear(128, num_classes)  # New fully connected layer
        self.dropout = nn.Dropout(0.5)
        self.layer_norm = nn.LayerNorm(hidden_size * 2 if bidirectional else hidden_size)

    def forward(self, x):
        rnn_outputs, _ = self.rnn(x)
        rnn_outputs = self.layer_norm(rnn_outputs)  # Apply layer normalization
        context_vector, _ = self.attention(rnn_outputs)
        x = torch.relu(self.fc1(self.dropout(context_vector)))
        x = torch.relu(self.fc2(self.dropout(x)))  # Apply ReLU to the output of the new layer
        x = self.fc3(x)
        return x

# Define the input size and number of classes
input_size = 6  # Number of features in the input data
hidden_size = 64  # Hidden size of the LSTM and attention mechanism
num_layers = 6  # Increase the number of LSTM layers
bidirectional = True  # Whether to use bidirectional LSTMs
num_classes = 4  # Number of classes (0, 1, 2, 3)

# Initialize the model
attention_model = EMGAttentionModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, num_classes=num_classes)


In [3]:
attention_model

EMGAttentionModel(
  (rnn): LSTM(6, 64, num_layers=6, batch_first=True, dropout=0.2, bidirectional=True)
  (attention): Attention(
    (W): Linear(in_features=128, out_features=128, bias=True)
    (v): Linear(in_features=128, out_features=1, bias=False)
  )
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

def train_model_attention(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, model_path='attention_model.pth'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Get the device
    model.to(device)  # Move model to GPU if available
    
    # Load the model state if available
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"Model loaded from {model_path}")
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(features)
            
            # Flatten the predictions to 2D tensor (batch_size * sequence_length, num_classes)
            outputs_flattened = outputs.view(-1, outputs.shape[-1])
            
            # Extract the last label in each sequence
            last_labels = labels[:, -1].long()  # Convert to torch.long
            
            # Calculate loss
            loss = criterion(outputs_flattened, last_labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == last_labels).sum().item()
            total_samples += labels.size(0)
        
        avg_train_loss = running_loss / len(train_loader)
        train_accuracy = correct_predictions / total_samples
        
        # Print the average training loss and accuracy
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation
        model.eval()  # Set the model to evaluation mode
        val_correct_predictions = 0
        val_total_samples = 0
        
        with torch.no_grad():
            for val_features, val_labels in val_loader:
                val_features, val_labels = val_features.to(device), val_labels.to(device)
                val_outputs = model(val_features)
                _, val_predicted = torch.max(val_outputs, 1)
                val_correct_predictions += (val_predicted == val_labels[:, -1].long()).sum().item()
                val_total_samples += val_labels.size(0)
        
        val_accuracy = val_correct_predictions / val_total_samples
        print(f"Validation Accuracy: {val_accuracy:.4f}")

        # Save the model after each epoch
        torch.save(model.state_dict(), "/kaggle/working/attention_modal.pth")
        print(f"Model saved ")

# # Define the input size and number of classes
# input_size = 6  # Number of features in the input data
# hidden_size = 64  # Hidden size of the LSTM and attention mechanism
# num_classes = 4  # Number of classes (0, 1, 2, 3)

# # Initialize the model
# attention_model = EMGAttentionModel(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)

# Assuming your EMGDataset, train_loader, val_loader are defined earlier

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = torch.optim.Adam(attention_model.parameters(), lr=0.001)

# Train the model and save it
train_model_attention(attention_model, train_loader, val_loader, criterion, optimizer, model_path='/kaggle/input/emg-fin-model-pth/attention_model.pth')


Model loaded from /kaggle/input/emg-fin-model-pth/attention_model.pth
Epoch 1/100, Train Loss: 0.5417, Train Accuracy: 0.8504
Validation Accuracy: 0.8667
Model saved 
Epoch 2/100, Train Loss: 0.3868, Train Accuracy: 0.8790
Validation Accuracy: 0.8792
Model saved 
Epoch 3/100, Train Loss: 0.3707, Train Accuracy: 0.8839
Validation Accuracy: 0.8729
Model saved 
Epoch 4/100, Train Loss: 0.3570, Train Accuracy: 0.8848
Validation Accuracy: 0.8667
Model saved 
Epoch 5/100, Train Loss: 0.2767, Train Accuracy: 0.9089
Validation Accuracy: 0.8750
Model saved 
Epoch 6/100, Train Loss: 0.2804, Train Accuracy: 0.9013
Validation Accuracy: 0.8708
Model saved 
Epoch 7/100, Train Loss: 0.2452, Train Accuracy: 0.9121
Validation Accuracy: 0.8625
Model saved 
Epoch 8/100, Train Loss: 0.2846, Train Accuracy: 0.8996
Validation Accuracy: 0.8479
Model saved 
Epoch 9/100, Train Loss: 0.2851, Train Accuracy: 0.9067
Validation Accuracy: 0.8604
Model saved 
Epoch 10/100, Train Loss: 0.2283, Train Accuracy: 0.9237


In [10]:
pip install pytorch_model_summary


Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from pytorch_model_summary import summary

# Define your model
input_size = 6
hidden_size = 64
num_layers = 6
bidirectional = True
num_classes = 4

attention_model = EMGAttentionModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, num_classes=num_classes)

# Generate model summary
model_summary = summary(attention_model, torch.zeros((32, 1000, input_size)))

# Print model summary
print(model_summary)


---------------------------------------------------------------------------------------------------
      Layer (type)                                    Output Shape         Param #     Tr. Param #
            LSTM-1     [32, 1000, 128], [12, 32, 64], [12, 32, 64]         533,504         533,504
       LayerNorm-2                                 [32, 1000, 128]             256             256
       Attention-3                           [32, 128], [32, 1000]          16,640          16,640
         Dropout-4                                       [32, 128]               0               0
          Linear-5                                       [32, 256]          33,024          33,024
          Linear-6                                       [32, 128]          32,896          32,896
          Linear-7                                         [32, 4]             516             516
Total params: 616,836
Trainable params: 616,836
Non-trainable params: 0
------------------------------------

In [8]:
# Define the input size and number of classes
input_size = 6  # Number of features in the input data
hidden_size = 64  # Hidden size of the LSTM and attention mechanism
num_layers = 6  # Increase the number of LSTM layers
bidirectional = True  # Whether to use bidirectional LSTMs
num_classes = 4  # Number of classes (0, 1, 2, 3)

# Initialize the model
attention_model = EMGAttentionModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, num_classes=num_classes)

# Load the model state dict
attention_model.load_state_dict(torch.load('/kaggle/input/emg-fin-model-pth/attention_model.pth'))


<All keys matched successfully>

In [6]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=ecd65cac712e1ddfa465b57625127bec48e3c7c51c0596b1d48457a96f17c32c
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [8]:
import torch
from torchviz import make_dot

# Initialize the model
input_size = 6  # Number of features in the input data
hidden_size = 64  # Hidden size of the LSTM and attention mechanism
num_layers = 6  # Increase the number of LSTM layers
bidirectional = True  # Whether to use bidirectional LSTMs
num_classes = 4  # Number of classes (0, 1, 2, 3)

attention_model = EMGAttentionModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, num_classes=num_classes)

# Generate a random input tensor with appropriate dimensions
batch_size = 32
sequence_length = 10
input_tensor = torch.randn(batch_size, sequence_length, input_size)

# Pass the input tensor through the model
output_tensor = attention_model(input_tensor)

# Generate visualization
dot = make_dot(output_tensor, params=dict(attention_model.named_parameters()))

# Specify the file path for saving the plot
file_path = "attention_model_plot.png"

# Save the plot to the specified file path
dot.render(filename=file_path, format='png')

print(f"Model plot saved as {file_path}")


Model plot saved as attention_model_plot.png


In [2]:
!pip install neurokit2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:0000:0100:01


In [3]:
from neurokit2 import *
import pandas as pd
fs = 500
def process_emg_signal(emg_signal):
    columns_to_ignore = ['Vbat', 'Trigger', 'AccX', 'AccY', 'AccZ', 'GyX', 'GyY', 'GyZ', 'fs', 'N', 'Timestamp']
    df_processed = emg_signal.drop(columns=columns_to_ignore)
    df_processed = df_processed.head(1000)
    emg_signals = []
    infos_temp = []
    for col_name, col_data in df_processed.items():
        emg_signal, info = emg_process(col_data, fs) 
        emg_signals.append(emg_signal)
        infos_temp.append(info)

    return emg_signals, infos_temp



In [57]:
flag = True
while flag:
    emg_signal = pd.DataFrame(pd.read_csv("/kaggle/input/emg-fin-csv/emg_dataset_final/down/down12.csv" , sep=';', skiprows=1, skipfooter=1, engine='python'))
    emg_signals , infos_temp = process_emg_signal(emg_signal)
#     emg_data = load_emg_signals_from_df_list(emg_signals)
    flag = False

/opt/conda/lib/python3.10/site-packages/neurokit2/emg/emg_activation.py:236: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_activity["EMG_Activity"][x] = 1
/opt/conda/lib/python3.10/site-packages/neurokit2/emg/emg_activation.py:241: Future

In [59]:
import torch
import torch.nn.functional as F
import pandas as pd

def real_time_prediction(data_frames, model):
    predictions = []

    for df in data_frames:
        # Convert the data frame to a PyTorch tensor
        emg_tensor = torch.tensor(df.values, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

        # Perform prediction
        with torch.no_grad():
            model.eval()  # Set the model to evaluation mode
            output = model(emg_tensor)

        # Apply softmax to get class probabilities
        probabilities = F.softmax(output, dim=1)
        
        # Get the predicted class
        _, predicted_class = torch.max(output, 1)

        predictions.append((probabilities, predicted_class.item()))

    return predictions

# Example usage:
# Assuming 'data_frames_list' is a list of pandas data frames containing real-time EMG data
predictions = real_time_prediction(emg_signals, attention_model)
for i, (probabilities, predicted_class) in enumerate(predictions):
    print(f"Prediction {i + 1}:")
    print("Predicted Class:", predicted_class)
    print("Class Probabilities:", probabilities)


Prediction 1:
Predicted Class: 1
Class Probabilities: tensor([[5.6258e-04, 9.9942e-01, 1.7388e-05, 2.3437e-09]])
Prediction 2:
Predicted Class: 1
Class Probabilities: tensor([[1.1765e-06, 9.9999e-01, 4.3443e-06, 9.4623e-08]])
Prediction 3:
Predicted Class: 1
Class Probabilities: tensor([[2.5169e-01, 7.3608e-01, 1.2165e-02, 7.1392e-05]])
Prediction 4:
Predicted Class: 1
Class Probabilities: tensor([[3.7898e-01, 6.1716e-01, 3.8517e-03, 8.7548e-06]])
Prediction 5:
Predicted Class: 0
Class Probabilities: tensor([[7.6438e-01, 1.9803e-02, 2.1561e-01, 2.0190e-04]])
Prediction 6:
Predicted Class: 1
Class Probabilities: tensor([[1.0824e-02, 9.8917e-01, 4.5010e-06, 6.6321e-09]])
Prediction 7:
Predicted Class: 1
Class Probabilities: tensor([[7.5049e-02, 9.2495e-01, 7.8908e-08, 2.0998e-11]])
Prediction 8:
Predicted Class: 1
Class Probabilities: tensor([[7.8172e-03, 9.9218e-01, 1.4887e-08, 4.0329e-12]])


In [ ]:
# # Define a function to load EMG signals from a list of DataFrames
# def load_emg_signals_from_df_list(df_list):
#     # Convert each DataFrame to a tensor and append to a list
#     tensor_list = []
#     for df in df_list:
#         tensor_data = torch.tensor(df.values, dtype=torch.float32)
#         tensor_list.append(tensor_data)
#     # Stack tensors along a new dimension to create a 3D tensor
#     tensor_data = torch.stack(tensor_list, dim=0)
#     return tensor_data

In [1]:
# def preprocess_input(dataframes):
#     # Convert each dataframe to a PyTorch tensor
#     tensors = [torch.tensor(df.values, dtype=torch.float32) for df in dataframes]
#     # Stack the tensors along the batch dimension
#     input_tensor = torch.stack(tensors)
#     # Reshape the tensor to match the model's input size
#     input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension
#     return input_tensor

densenet + resnet